In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import pandas as pd
import numpy as np
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from difflib import SequenceMatcher

In [29]:
driver = webdriver.Chrome()
driver.get("https://services.livpartner.com/ci_service/index.php")
driver.maximize_window()

In [30]:
# Login Livpure Dashboard
email_input = driver.find_element(By.ID,"user_name")
email_input.send_keys("1210193577")
password_input = driver.find_element(By.ID,"user_pass")
password_input.send_keys("Apexenterprises@128")
submit_button = driver.find_element(By.NAME,"login")
submit_button.click()
time.sleep(40)
submit_button = driver.find_element(By.NAME,"login")
submit_button.click()

In [31]:
# import os
# file_datetime = datetime.now().strftime('%d-%m-%Y_%H-%M')
# folder_name = f"open_cases/open_cases_{file_datetime}"
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)


In [32]:
# Data Scraping
open_cases_url = "https://services.livpartner.com/ci_service/index.php/Case_assign?type=ro"
# open_cases_url = "https://services.livpartner.com/ci_service/index.php/my_cases/pending_case?type=ro"
driver.get(open_cases_url)

dropdown = driver.find_element(By.XPATH,"/html/body/section/section/div/div[2]/div/div[1]/label/select[@name = 'myTable_length']")
select_object = Select(dropdown)
select_object.select_by_index(3)

last_page_xpath = '/html/body/section/section/div/div[2]/div/div[4]/span/a[last()]'
last_page = driver.find_element(By.XPATH, last_page_xpath).text
next_button_id = 'myTable_next'
file_datetime = datetime.now().strftime('%d-%m-%Y_%H-%M')
table_data_list = []
count = 0

while count < int(last_page):
  
  table_data = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "myTable")))
  table_html = table_data.get_attribute('outerHTML')
  df_list = pd.read_html(table_html)
  table_data_list.append(df_list[0])
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  time.sleep(4)
  next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, next_button_id)))
  next_button.click()
  time.sleep(5)
  count = count + 1

final_df = pd.concat(table_data_list, ignore_index=True)
excel_file_path = f"open_cases_{file_datetime}.xlsx"
final_df["Cust Address"] = final_df["Cust Address"].tolist()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/section/section/div/div[2]/div/div[1]/label/select[@name = 'myTable_length']"}
  (Session info: chrome=115.0.5790.171); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0121A813+48355]
	(No symbol) [0x011AC4B1]
	(No symbol) [0x010B5358]
	(No symbol) [0x010E09A5]
	(No symbol) [0x010E0B3B]
	(No symbol) [0x0110E232]
	(No symbol) [0x010FA784]
	(No symbol) [0x0110C922]
	(No symbol) [0x010FA536]
	(No symbol) [0x010D82DC]
	(No symbol) [0x010D93DD]
	GetHandleVerifier [0x0147AABD+2539405]
	GetHandleVerifier [0x014BA78F+2800735]
	GetHandleVerifier [0x014B456C+2775612]
	GetHandleVerifier [0x012A51E0+616112]
	(No symbol) [0x011B5F8C]
	(No symbol) [0x011B2328]
	(No symbol) [0x011B240B]
	(No symbol) [0x011A4FF7]
	BaseThreadInitThunk [0x76176BA9+2

In [ ]:
area_list = pd.read_excel("area_list.xlsx")
area_list = area_list['area_list'].tolist()

In [ ]:
def find_matching_job_areas(address):
    matching_areas = []
    address_words = set(address.lower().split())
    
    for area in area_list:
        area_words = set(area.lower().split())
        common_words = address_words.intersection(area_words)
        similarity_ratio = len(common_words) / len(area_words)
        
        if similarity_ratio >= 0.6:
            matching_areas.append(area)
    
    return matching_areas if matching_areas else ["Not Found"]

# Apply the function to create a new column 'area_list_match'
final_df["job_area_match"] = final_df["Cust Address"].apply(find_matching_job_areas)
final_df.to_excel(excel_file_path, index=False)


In [ ]:

df =  pd.read_excel(excel_file_path)
model_value_counts = df['Product Model No'].value_counts()
model_name = df['Product Model No'].unique()
df.loc[df['Case Type'].str.startswith('PM_Call'), 'Case Type'] = 'PM_Call'
case_count = df['Case Type'].unique()
# Create a Pivot Table to get the data in the right format for the stacked bar chart
model_job_pivot = df.pivot_table(index=df['Product Model No'], columns=df['Case Type'], aggfunc='size', fill_value=0)
# Create the stacked bar chart using Plotly Express
fig1 = px.bar(model_job_pivot, x=model_job_pivot.index, y=model_job_pivot.columns, title='Open Cases Count of Model and Job', 
              labels={'x': 'model_name', 'y': 'Count', 'color': df['Case Type']}, barmode='stack', 
              color_discrete_map={'PM_Call': 'green', 'Install_Req': 'brown', 'Service_Complaint': 'blue'}
              )
fig1.show()
# fig1 = px.bar(x=model_name, y=model_value_counts, labels={'Product Model No': 'Unique Values', 'y': 'Count'})
# fig2 = px.bar(x=model_name, y=case_count, labels={'Product Model No': 'Unique Values', 'y': 'Count'})
# fig = sp.make_subplots(rows=1, cols=1, subplot_titles=("Open_Cases"))
# fig.add_trace(fig1.data[*], row=1, col=1)
# fig.add_trace(fig2.data[0], row=2, col=1)
# print(model_name,model_value_counts,case_count )

In [ ]:
from datetime import datetime
df['Case Create Date'] = pd.to_datetime(df['Case Create Date'])
current_datetime = datetime.now()
time_difference = current_datetime - df['Case Create Date']
df['time_from_now'] = time_difference.dt.days.astype(str) + " days " + (time_difference.dt.seconds // 3600).astype(str) + " hours"
excel_file_path_aging = f"open_aging_{file_datetime}.xlsx"
df.to_excel(excel_file_path_aging, index=False)

In [ ]:
service_complaint_df = df[df['Case Type'] == 'Service_Complaint'].copy()
service_complaint_df.sort_values(by='time_from_now', ascending=False, inplace=True)
excel_file_path_complaints = f"open_cases_complaints_{file_datetime}.xlsx"
service_complaint_df.to_excel(excel_file_path_complaints, index=False)

In [ ]:
Install_Req_df = df[df['Case Type'] == 'Install_Req'].copy()
Install_Req_df.sort_values(by='time_from_now', ascending=False, inplace=True)
excel_file_path_installation = f"open_cases_installation_{file_datetime}.xlsx"
Install_Req_df.to_excel(excel_file_path_installation, index=False)

In [ ]:
PM_Call_df = df[df['Case Type'] == 'PM_Call'].copy()
PM_Call_df.sort_values(by='time_from_now', ascending=False, inplace=True)
PM_Call_df['time_from_now'] = pd.to_timedelta(PM_Call_df['time_from_now'])
filtered_df = PM_Call_df[PM_Call_df['time_from_now'] < pd.Timedelta(days=6)]
excel_file_path_pm = f"open_cases_pm_6days_{file_datetime}.xlsx"
filtered_df.to_excel(excel_file_path_pm, index=False)